In [1]:
!pip3 install -q -U transformers==4.38.2
!pip3 install -q -U datasets==2.18.0
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.9.0
!pip3 install -q -U trl==0.7.11
!pip3 install -q -U accelerate==0.27.2

## 1. 개발환경설정

### 1.1 Import Modules

In [2]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

### 1.2 Huggingface 로그인

In [3]:
from huggingface_hub import notebook_login
notebook_login()

## 2. Dataset 준비
판례 요약 모델을 만드는 것이 목적이므로 AI허브의 '문서요약 텍스트' 데이터의 판례 요약데이터를 가져옴

### 2.1 Load dataset

In [4]:
import json

with open('./판례요약데이터/Training/법률_train_original/train_original.json', 'r') as f:
    dataset = json.load(f)

In [ ]:
type(dataset)

dict

### 2.2 데이터셋 탐색

In [ ]:
dataset['documents'][0]['text']

[[{'index': 0,
   'sentence': '원고가 소속회사의 노동조합에서 분규가 발생하자 노조활동을 구실로 정상적인 근무를 해태하고,',
   'highlight_indices': ''},
  {'index': 1, 'sentence': '노조조합장이 사임한 경우,', 'highlight_indices': ''},
  {'index': 2,
   'sentence': '노동조합규약에 동 조합장의 직무를 대행할 자를 규정해 두고 있음에도 원고 자신이 주동하여 노조자치수습대책위원회를 구성하여 그 위원장으로 피선되어 근무시간중에도 노조활동을 벌여 운수업체인 소속회사의 업무에 지장을 초래하고',
   'highlight_indices': '8,9;68,69'},
  {'index': 3,
   'sentence': '종업원들에게도 나쁜 영향을 끼쳐 소속회사가 취업규칙을 위반하고',
   'highlight_indices': ''},
  {'index': 4,
   'sentence': '고의로 회사업무능률을 저해하였으며 회사업무상의 지휘명령에 위반하였음을 이유로 원고를 징계해고 하였다면,',
   'highlight_indices': '0,3'},
  {'index': 5,
   'sentence': '이는 원고의 노동조합 활동과는 관계없이 회사취업규칙에 의하여 사내질서를 유지하기 위한 사용자 고유의 징계권에 기하여 이루어진 정당한 징계권의 행사로 보아야 한다.',
   'highlight_indices': '17,21'}]]

## 3. Gemma 모델의 한국어 요약 테스트
Google이 제공하는 Gemma 모델은 2가지 타입, base 모델과 IT 모델(Instruction Tuned) 모델을 포함합니다.

이 두 모델의 주된 차이는 IT 모델이 대화 형식의 상호작용에 최적화되어 사용자의 의도를 더 정확하게 파악하여 보다 정제된 답변을 제공한다는 것입니다.

이는 ChatGPT와 유사한 경험을 제공하며, 질문에 대한 응답에서 높은 수준의 맥락 이해와 적합성을 보입니다.

반면 base 모델은 이러한 튜닝 작업을 거치지 않은 원본의 모델이라서 출력 품질로 보면 IT 모델보다 떨어지지만

다른 방식으로 Instruct tuning을 거쳐 사용자의 목적에 더 적합한 모델을 만들어 낼 수 있는 자유도가 있습니다.

우리는 요약 finetuned 모델을 만들기 위해 사전 tuning된 google/gemma-2b-it 모델을 사용해 볼 것입니다.

In [ ]:
Base_model = "google/gemma-2b-it"

model = AutoModelForCausalLM.from_pretrained(Base_model, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(Base_model, add_special_tokens=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

### 3.2 Gemma-it의 프롬프트 형식
```
<bos><start_of_turn>user
Write a hello world program<end_of_turn>
<start_of_turn>model
```
프롬프트 형식을 살펴보면 전형적인 chat template의 형식을 가지고 있음을 확인 할 수 있습니다.

<start_of_turn> 이라는 구분자를 통해 대화 턴의 시작을 구분하며, 대화의 끝은<end_of_turn>구분자로 마무리 짓고 있습니다.

또한 대화자는 user와 model로 지정됨을 알 수 있습니다.

그럼 이제 부터 우리는 모델에 질문을 할 때 위와 같은 형식으로 프롬프트를 만들어야 합니다.

prompt 템플릿을 만들어서 사용해도 되겠지만, huggingface 는 보다 쉬운 방법을 제공 합니다.

tokenizer의 apply.chat_template 메소드가 바로 그것인데요, 사용 방법은 다음과 같습니다.

프롬프트 형식을 살펴보면 전형적인 chat template의 형식을 가지고 있음을 확인 할 수 있습니다.

<start_of_turn> 이라는 구분자를 통해 대화 턴의 시작을 구분하며, 대화의 끝은<end_of_turn>구분자로 마무리 짓고 있습니다.

또한 대화자는 user와 model로 지정됨을 알 수 있습니다.

그럼 이제 부터 우리는 모델에 질문을 할 때 위와 같은 형식으로 프롬프트를 만들어야 합니다.

prompt 템플릿을 만들어서 사용해도 되겠지만, huggingface 는 보다 쉬운 방법을 제공 합니다.

tokenizer의 apply.chat_template 메소드가 바로 그것인데요, 사용 방법은 다음과 같습니다.
```
messages = [
    {"role": "user",
     "content": "...."}
     ]

prompt = pipeline.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
```

In [ ]:
dataset['documents'][0]['text'][0][0]

{'index': 0,
 'sentence': '원고가 소속회사의 노동조합에서 분규가 발생하자 노조활동을 구실로 정상적인 근무를 해태하고,',
 'highlight_indices': ''}

In [ ]:
doc = "직권으로 판단한다.  1. 기록에 의하면, 피고인에 대한 당원의 소송기록접수통지서가 2005. 3. 3. 피고인에게 송달되었음에도 우편집배원의 착오로 우편송달통지서상 송달일자가 '2005. 2. 28.'로 기재된 사실, 피고인의 상고이유서는 그 제출기간 내인 2005. 3. 22. 당원에 접수된 사실, 그런데 당원은 착오에 의한 송달일자를 진실한 것으로 믿고 2005. 3. 25. 피고인이 법정기간 내에 상고이유서를 제출하지 아니하였다는 이유로 피고인의 상고를 기각하는 결정을 한 사실을 알 수 있으므로, 형사소송법 제400조에 의하여 그 내용에 오류가 있음이 분명한 당원의 위 결정을 정정하기로 하여 다음과 같이 판결한다. 2. 상고이유를 판단한다.  기록에 의하여 살펴보면, 원심이 판시 각 증거에 의하여 피고인이 피해자 홍만기와의 임대차계약 체결 당시 피해자에게 보증금을 반환할 의사나 능력이 없었던 사실, 피고인이 공소사실 기재와 같이 법원을 기망하여 500만 원의 배당금을 편취한 사실이 인정된다고 판단한 것은 정당하고, 상고이유의 주장과 같은 채증법칙 위배 내지 심리미진으로 인한 사실오인의 위법이 있다고 할 수 없으며, 한편 벌금형이 선고된 이 사건에서 원심의 형량이 너무 무겁다는 주장은 적법한 상고이유가 될 수 없다.  그러므로 피고인의 상고를 기각하기로 하여 관여 대법관의 일치된 의견으로 주문과 같이 판결한다. 대법관 이강국(재판장) 유지담(주심) 배기원 김용담 "

In [ ]:
pipe = pipeline("text-generation", model = model, tokenizer=tokenizer, max_new_tokens=512)

In [ ]:
messages = [
    {
        "role": "user",
        "content": "다음 글을 요약해주세요:\n\n{}".format(doc)
    }
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [ ]:
prompt

"<bos><start_of_turn>user\n다음 글을 요약해주세요:\n\n직권으로 판단한다.  1. 기록에 의하면, 피고인에 대한 당원의 소송기록접수통지서가 2005. 3. 3. 피고인에게 송달되었음에도 우편집배원의 착오로 우편송달통지서상 송달일자가 '2005. 2. 28.'로 기재된 사실, 피고인의 상고이유서는 그 제출기간 내인 2005. 3. 22. 당원에 접수된 사실, 그런데 당원은 착오에 의한 송달일자를 진실한 것으로 믿고 2005. 3. 25. 피고인이 법정기간 내에 상고이유서를 제출하지 아니하였다는 이유로 피고인의 상고를 기각하는 결정을 한 사실을 알 수 있으므로, 형사소송법 제400조에 의하여 그 내용에 오류가 있음이 분명한 당원의 위 결정을 정정하기로 하여 다음과 같이 판결한다. 2. 상고이유를 판단한다.  기록에 의하여 살펴보면, 원심이 판시 각 증거에 의하여 피고인이 피해자 홍만기와의 임대차계약 체결 당시 피해자에게 보증금을 반환할 의사나 능력이 없었던 사실, 피고인이 공소사실 기재와 같이 법원을 기망하여 500만 원의 배당금을 편취한 사실이 인정된다고 판단한 것은 정당하고, 상고이유의 주장과 같은 채증법칙 위배 내지 심리미진으로 인한 사실오인의 위법이 있다고 할 수 없으며, 한편 벌금형이 선고된 이 사건에서 원심의 형량이 너무 무겁다는 주장은 적법한 상고이유가 될 수 없다.  그러므로 피고인의 상고를 기각하기로 하여 관여 대법관의 일치된 의견으로 주문과 같이 판결한다. 대법관 이강국(재판장) 유지담(주심) 배기원 김용담<end_of_turn>\n<start_of_turn>model\n"

### 3.3 Gemma-it 추론

In [ ]:
outputs= pipe(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)

In [ ]:
print(outputs[0]['generated_text'][len(prompt):])

**요약:**

* 피고인의 소송기록접수통지서는 2005. 3월 28일로 기재되었지만, 착오로 우편집배원의 착오로 송달일자는 2005. 2. 28일로 기록되어 피고인의 상고를 기각할 수 있다.

**결론:**

* 피고인의 상고를 기각한다.


## 4. Gemma 파인튜닝
AI허브의 판례요약데이터를 가지고 Gemma 모델을 파인튜닝 해보자
### 4.1 학습용 프롬프트 조정
모델에게 "이렇게 질문하면 이렇게 대답해야해"를 가르쳐야함 \
쓸데 없는 내용들이 계속 이어져 출력이 될 수 있어서 이러한 내용들이 출력되지 않도록 학습 데이터에 문장의 끝이라는 <eos> token을 명시적으로 붙여서 학습하게 되면 모델은 요약 내용만 출력하고 더 이상 생성을 멈출 것
```
def generate_prompt(example):
    prompt_list = []
    for i in range(len(example['document'])):
        prompt_list.append(r"""<bos><start_of_turn>user
다음 글을 요약해주세요:

{}<end_of_turn>
<start_of_turn>model
{}<end_of_turn><eos>""".format(example['document'][i], example['summary'][i]))
    return prompt_list
```
결과:
```
<bos><start_of_turn>user
다음 글을 요약해주세요:

{본문} <end_of_turn>
<start_of_turn>model
{요약문} <end_of_turn><eos>
```


실제 train data를 넣어서 테스트해보자\
그 전에 train data를 사용할 수 있도록 전처리

In [ ]:
# 판례내용
dataset['documents'][0]['text']

[[{'index': 0,
   'sentence': '원고가 소속회사의 노동조합에서 분규가 발생하자 노조활동을 구실로 정상적인 근무를 해태하고,',
   'highlight_indices': ''},
  {'index': 1, 'sentence': '노조조합장이 사임한 경우,', 'highlight_indices': ''},
  {'index': 2,
   'sentence': '노동조합규약에 동 조합장의 직무를 대행할 자를 규정해 두고 있음에도 원고 자신이 주동하여 노조자치수습대책위원회를 구성하여 그 위원장으로 피선되어 근무시간중에도 노조활동을 벌여 운수업체인 소속회사의 업무에 지장을 초래하고',
   'highlight_indices': '8,9;68,69'},
  {'index': 3,
   'sentence': '종업원들에게도 나쁜 영향을 끼쳐 소속회사가 취업규칙을 위반하고',
   'highlight_indices': ''},
  {'index': 4,
   'sentence': '고의로 회사업무능률을 저해하였으며 회사업무상의 지휘명령에 위반하였음을 이유로 원고를 징계해고 하였다면,',
   'highlight_indices': '0,3'},
  {'index': 5,
   'sentence': '이는 원고의 노동조합 활동과는 관계없이 회사취업규칙에 의하여 사내질서를 유지하기 위한 사용자 고유의 징계권에 기하여 이루어진 정당한 징계권의 행사로 보아야 한다.',
   'highlight_indices': '17,21'}]]

In [ ]:
dataset['documents'][0]['text'][0][0]['sentence']

'원고가 소속회사의 노동조합에서 분규가 발생하자 노조활동을 구실로 정상적인 근무를 해태하고,'

In [ ]:
# 판례 요약
dataset['documents'][0]['abstractive']

['원고가  주동하여 회사업무능률을 저해하고 회사업무상의 지휘명령에 위반하였다면 이에 따른 징계해고는 사내질서를 유지하기 위한 사용자 고유의 정당한 징계권의 행사로 보아야 한다.']

In [9]:
train_data = {}
text = []
summary = []
for i in range(len(dataset['documents'])):
  temp = ''
  for sen in dataset['documents'][i]['text'][0]:
    temp = temp + sen['sentence']
  text.append(temp)
print(text[0])

for i in range(len(dataset['documents'])):
  summary.append(*dataset['documents'][i]['abstractive'])
print(summary[0])

train_data['text'] = text
train_data['summary'] = summary

원고가 소속회사의 노동조합에서 분규가 발생하자 노조활동을 구실로 정상적인 근무를 해태하고,노조조합장이 사임한 경우,노동조합규약에 동 조합장의 직무를 대행할 자를 규정해 두고 있음에도 원고 자신이 주동하여 노조자치수습대책위원회를 구성하여 그 위원장으로 피선되어 근무시간중에도 노조활동을 벌여 운수업체인 소속회사의 업무에 지장을 초래하고종업원들에게도 나쁜 영향을 끼쳐 소속회사가 취업규칙을 위반하고고의로 회사업무능률을 저해하였으며 회사업무상의 지휘명령에 위반하였음을 이유로 원고를 징계해고 하였다면,이는 원고의 노동조합 활동과는 관계없이 회사취업규칙에 의하여 사내질서를 유지하기 위한 사용자 고유의 징계권에 기하여 이루어진 정당한 징계권의 행사로 보아야 한다.
원고가  주동하여 회사업무능률을 저해하고 회사업무상의 지휘명령에 위반하였다면 이에 따른 징계해고는 사내질서를 유지하기 위한 사용자 고유의 정당한 징계권의 행사로 보아야 한다.


In [10]:
import pandas as pd
from datasets import Dataset

train_data = pd.DataFrame(train_data)
train_data = Dataset.from_pandas(train_data)

In [ ]:
train_data[0]

{'text': '원고가 소속회사의 노동조합에서 분규가 발생하자 노조활동을 구실로 정상적인 근무를 해태하고,노조조합장이 사임한 경우,노동조합규약에 동 조합장의 직무를 대행할 자를 규정해 두고 있음에도 원고 자신이 주동하여 노조자치수습대책위원회를 구성하여 그 위원장으로 피선되어 근무시간중에도 노조활동을 벌여 운수업체인 소속회사의 업무에 지장을 초래하고종업원들에게도 나쁜 영향을 끼쳐 소속회사가 취업규칙을 위반하고고의로 회사업무능률을 저해하였으며 회사업무상의 지휘명령에 위반하였음을 이유로 원고를 징계해고 하였다면,이는 원고의 노동조합 활동과는 관계없이 회사취업규칙에 의하여 사내질서를 유지하기 위한 사용자 고유의 징계권에 기하여 이루어진 정당한 징계권의 행사로 보아야 한다.',
 'summary': '원고가  주동하여 회사업무능률을 저해하고 회사업무상의 지휘명령에 위반하였다면 이에 따른 징계해고는 사내질서를 유지하기 위한 사용자 고유의 정당한 징계권의 행사로 보아야 한다.'}

In [11]:
def generate_prompt(example):
    prompt_list = []
    for i in range(len(example['text'])):
        prompt_list.append(r"""<bos><start_of_turn>user
다음 글을 요약해주세요:

{}<end_of_turn>
<start_of_turn>model
{}<end_of_turn><eos>""".format(example['text'][i], example['summary'][i]))
    return prompt_list

In [ ]:
print(generate_prompt(train_data)[0])

<bos><start_of_turn>user
다음 글을 요약해주세요:

원고가 소속회사의 노동조합에서 분규가 발생하자 노조활동을 구실로 정상적인 근무를 해태하고,노조조합장이 사임한 경우,노동조합규약에 동 조합장의 직무를 대행할 자를 규정해 두고 있음에도 원고 자신이 주동하여 노조자치수습대책위원회를 구성하여 그 위원장으로 피선되어 근무시간중에도 노조활동을 벌여 운수업체인 소속회사의 업무에 지장을 초래하고종업원들에게도 나쁜 영향을 끼쳐 소속회사가 취업규칙을 위반하고고의로 회사업무능률을 저해하였으며 회사업무상의 지휘명령에 위반하였음을 이유로 원고를 징계해고 하였다면,이는 원고의 노동조합 활동과는 관계없이 회사취업규칙에 의하여 사내질서를 유지하기 위한 사용자 고유의 징계권에 기하여 이루어진 정당한 징계권의 행사로 보아야 한다.<end_of_turn>
<start_of_turn>model
원고가  주동하여 회사업무능률을 저해하고 회사업무상의 지휘명령에 위반하였다면 이에 따른 징계해고는 사내질서를 유지하기 위한 사용자 고유의 정당한 징계권의 행사로 보아야 한다.<end_of_turn><eos>


### 4.2 QLoRA 설정
- LoRA(Low-Rank Adaptation) 기법: 기존 모델의 파라미터를 직접 수정하는 대신, 추가적인 작은 파라미터를 도입하여 모델의 출력을 조절하는 방식
  - 크기를 크게 늘리지 않으면서도, 특정 작업에 모델을 효과적으로 최적화할 수 있는 장점 덕분에 LLM의 파인튜닝에 많이 사용됨
- QLoRA: Pre-Trained 모델 자체도 4-bit로 양자화하고, 그 위에 LoRA 기법을 적용하여 파인튜닝
  - 양자화 (Quantization) 과정은 모델의 메모리 사용량을 대폭 줄여주어, 더 큰 모델을 더 적은 리소스로 학습 시킬 수 있게 만들어 줌

- LoRA와 QLoRA의 자세한 내용은 아래를 참고

LoRA : https://huggingface.co/docs/peft/conceptual_guides/lora

LoRAConfig : https://huggingface.co/docs/peft/main/en/package_reference/tuners#peft.LoraConfig

QLoRA : https://huggingface.co/blog/4bit-transformers-bitsandbytes

QLoRA를 사용하기 위한 설정은 다음과 같이 정의

In [12]:
lora_config = LoraConfig(
    r=6,
    lora_alpha = 8,
    lora_dropout = 0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

QLoRA 적용해서 모델을 로드

In [13]:
Base_model = "google/gemma-2b-it"

model = AutoModelForCausalLM.from_pretrained(Base_model, device_map="auto", quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(Base_model, add_special_token=True)
tokenizer.padding_side = 'right'

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

### 4.3 Trainer 실행
학습은 Huggingface Transfomers의 Trainer 클래스 대신 SFTTrainer 클래스를 사용할 것입니다.

SFTTrainer (Supervised Fine-tuning Trainer)는 Trainer 클래스에 비해 대화형 또는 Instruction 포멧의 데이터셋을 다루는데 보다 효율적인 방법을 제공합니다.

formatting_prompts_func이 그러한 기능들중 하나입니다.

In [15]:
trainer = SFTTrainer(
    model = model,
    train_dataset = train_data,
    max_seq_length=512,
    args=TrainingArguments(
        output_dir="outputs",
        max_steps=3000,
        per_device_train_batch_size = 1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_8bit",
        warmup_steps=0.03,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=100,
        push_to_hub = False,
        report_to='none',
    ),
    peft_config=lora_config,
    formatting_func=generate_prompt,
)

Map:   0%|          | 0/24329 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:294: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [16]:
trainer.train()

Step,Training Loss
100,2.249800
200,1.831100
300,1.772500
400,1.740600
500,1.683600
600,1.680700
700,1.644500
800,1.628600
900,1.610300
1000,1.591200


TrainOutput(global_step=3000, training_loss=1.589129404703776, metrics={'train_runtime': 7377.2617, 'train_samples_per_second': 1.627, 'train_steps_per_second': 0.407, 'total_flos': 5.456321254018253e+16, 'train_loss': 1.589129404703776, 'epoch': 0.49})

### 4.4 Finetuned Model 저장

In [17]:
Adapter_model = "lora_adapter"

trainer.model.save_pretrained(Adapter_model)

In [18]:
!ls -alh lora_adapter

total 29M
drwxr-xr-x 2 root root 4.0K May 21 07:45 .
drwxr-xr-x 1 root root 4.0K May 21 07:45 ..
-rw-r--r-- 1 root root  690 May 21 07:45 adapter_config.json
-rw-r--r-- 1 root root  29M May 21 07:45 adapter_model.safetensors
-rw-r--r-- 1 root root 5.0K May 21 07:45 README.md


In [19]:
# LoRA 학습된 weight을 원래 gemma-7b-it 모델과 합쳐 하나의 finetuned 모델로 만들기
model = AutoModelForCausalLM.from_pretrained(Base_model, device_map='auto', torch_dtype=torch.float16)
model = PeftModel.from_pretrained(model, Adapter_model, device_map='auto', torch_dtype=torch.float16)

model = model.merge_and_unload()
model.save_pretrained('gemma-2b-it-sum-ko')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [20]:
!ls -alh ./gemma-2b-it-sum-ko

total 4.7G
drwxr-xr-x 2 root root 4.0K May 21 07:46 .
drwxr-xr-x 1 root root 4.0K May 21 07:45 ..
-rw-r--r-- 1 root root  662 May 21 07:45 config.json
-rw-r--r-- 1 root root  132 May 21 07:45 generation_config.json
-rw-r--r-- 1 root root 4.7G May 21 07:46 model-00001-of-00002.safetensors
-rw-r--r-- 1 root root  65M May 21 07:46 model-00002-of-00002.safetensors
-rw-r--r-- 1 root root  14K May 21 07:46 model.safetensors.index.json


## 5. Gemma 한국어 요약 모델 추론
### 5.1 Fine-tuned 모델 로드

In [6]:
Base_model = "google/gemma-2b-it"
Finetuned_model = "./00. Data/finetuned_gemma/gemma-2b-it-sum-ko"

finetuned_model = AutoModelForCausalLM.from_pretrained(Finetuned_model, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(Base_model, add_special_token=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


### 5.2 Fine-tuned 모델 추론

In [7]:
pipe_finetuned = pipeline("text-generation", model=finetuned_model, tokenizer=tokenizer, max_new_tokens=512)

In [8]:
with open('./판례요약데이터/Validation/법률_valid_original/valid_original.json', 'r') as f:
    dataset_valid = json.load(f)

In [9]:
valid_data = {}
text = []
summary = []
for i in range(len(dataset_valid['documents'])):
  temp = ''
  for j in range(len(dataset_valid['documents'][i]['text'])):
    for s in dataset_valid['documents'][i]['text'][j]:
      temp = temp + s['sentence']
  text.append(temp)
print(text[0])

for i in range(len(dataset_valid['documents'])):
  summary.append(*dataset_valid['documents'][i]['abstractive'])
print(summary[0])

valid_data['text'] = text
valid_data['summary'] = summary

[1] 취소소송은 처분 등이 있음을 안 날부터 90일 이내에 제기하여야 하고,처분 등이 있은 날부터 1년을 경과하면제기하지 못하며( 행정소송법 제20조 제1항, 제2항), 청구취지를 변경하여 구 소가 취하되고 새로운 소가 제기된 것으로 변경되었을 때에 새로운 소에 대한 제소기간의 준수 등은 원칙적으로 소의 변경이 있은 때를 기준으로 하여야 한다.[2] 일반적으로 행정처분에 효력기간이 정하여져 있는 경우에는 그 기간의 경과로 그 행정처분의 효력은 상실되며,다만 허가에 붙은 기한이 그 허가된 사업의 성질상 부당하게 짧은 경우에는 이를 그 허가 자체의 존속기간이 아니라 그 허가조건의 존속기간으로 보아 그 기한이 도래함으로써 그 조건의 개정을 고려한다는 뜻으로 해석할 수 있다.[3] 사도개설허가에서 정해진 공사기간 내에 사도로 준공검사를 받지 못한 경우,이 공사기간을 사도개설허가 자체의 존속기간(유효기간)으로 볼 수 없다는 이유로 사도개설허가가 당연히 실효되는 것은 아니라고 한 사례.
취소소송은 처분 등이 있다는 것을 안 때로부터 90일 이내에 제기하여야 하고, 행정처분에서의 허가에 붙은 기한이 부당하게 짧은 경우에는 이를 허가조건 존속기간으로 보아서 그 기한의 도래로 조건 개정을 고려한다고 해석할 수 있기에, 사도개설허가의 준공검사를 받지 못한 것은 사도개설허가 자체의 존속기간으로 볼 수 없다는 까닭으로 이것이 실효되는 것은 아니다.


In [11]:
doc_val = valid_data['text'][0]
print(doc_val)

[1] 취소소송은 처분 등이 있음을 안 날부터 90일 이내에 제기하여야 하고,처분 등이 있은 날부터 1년을 경과하면제기하지 못하며( 행정소송법 제20조 제1항, 제2항), 청구취지를 변경하여 구 소가 취하되고 새로운 소가 제기된 것으로 변경되었을 때에 새로운 소에 대한 제소기간의 준수 등은 원칙적으로 소의 변경이 있은 때를 기준으로 하여야 한다.[2] 일반적으로 행정처분에 효력기간이 정하여져 있는 경우에는 그 기간의 경과로 그 행정처분의 효력은 상실되며,다만 허가에 붙은 기한이 그 허가된 사업의 성질상 부당하게 짧은 경우에는 이를 그 허가 자체의 존속기간이 아니라 그 허가조건의 존속기간으로 보아 그 기한이 도래함으로써 그 조건의 개정을 고려한다는 뜻으로 해석할 수 있다.[3] 사도개설허가에서 정해진 공사기간 내에 사도로 준공검사를 받지 못한 경우,이 공사기간을 사도개설허가 자체의 존속기간(유효기간)으로 볼 수 없다는 이유로 사도개설허가가 당연히 실효되는 것은 아니라고 한 사례.


프롬프트 구성

In [12]:
messages = [
    {
        "role": "user",
        "content": "다음 글을 요약해주세요:\n\n{}".format(doc_val)
    }
]
prompt = pipe_finetuned.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [13]:
outputs = pipe_finetuned(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(prompt):])

취소소송은 처분 등이 있음을 안 날부터 90일 이내에 제기하여야 하는데, 허가에 붙은 기한이 그 허가된 사업의 성질상 부당하게 짧은 경우에는 이를 그 허가 자체의 존속기간이 아니라 그 허가조건의 존속기간으로 보아 그 기한이 도래함으로써 그 조건의 개정을 고려한다는 뜻으로 해석할 수 있다.


In [14]:
valid_data['summary'][0]

'취소소송은 처분 등이 있다는 것을 안 때로부터 90일 이내에 제기하여야 하고, 행정처분에서의 허가에 붙은 기한이 부당하게 짧은 경우에는 이를 허가조건 존속기간으로 보아서 그 기한의 도래로 조건 개정을 고려한다고 해석할 수 있기에, 사도개설허가의 준공검사를 받지 못한 것은 사도개설허가 자체의 존속기간으로 볼 수 없다는 까닭으로 이것이 실효되는 것은 아니다.'